# Initial Overview and Cleaning

## Imports

In [74]:
import pandas as pd
import numpy as np
import glob 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import dates as mdates
import matplotlib.ticker as mticks
from sklearn.compose import ColumnTransformer
import holidays
import datetime as dt

## Functions

## Importing Data & Initial Inspection

In [2]:
# Using glob to get all filepaths that match the pattern (*=wildcard)
glob = sorted(glob.glob("Data/Raw-Data/Chicago-Crime_2*.csv"))
glob

['Data/Raw-Data\\Chicago-Crime_2001.csv',
 'Data/Raw-Data\\Chicago-Crime_2002.csv',
 'Data/Raw-Data\\Chicago-Crime_2003.csv',
 'Data/Raw-Data\\Chicago-Crime_2004.csv',
 'Data/Raw-Data\\Chicago-Crime_2005.csv',
 'Data/Raw-Data\\Chicago-Crime_2006.csv',
 'Data/Raw-Data\\Chicago-Crime_2007.csv',
 'Data/Raw-Data\\Chicago-Crime_2008.csv',
 'Data/Raw-Data\\Chicago-Crime_2009.csv',
 'Data/Raw-Data\\Chicago-Crime_2010.csv',
 'Data/Raw-Data\\Chicago-Crime_2011.csv',
 'Data/Raw-Data\\Chicago-Crime_2012.csv',
 'Data/Raw-Data\\Chicago-Crime_2013.csv',
 'Data/Raw-Data\\Chicago-Crime_2014.csv',
 'Data/Raw-Data\\Chicago-Crime_2015.csv',
 'Data/Raw-Data\\Chicago-Crime_2016.csv',
 'Data/Raw-Data\\Chicago-Crime_2017.csv',
 'Data/Raw-Data\\Chicago-Crime_2018.csv',
 'Data/Raw-Data\\Chicago-Crime_2019.csv',
 'Data/Raw-Data\\Chicago-Crime_2020.csv',
 'Data/Raw-Data\\Chicago-Crime_2021.csv',
 'Data/Raw-Data\\Chicago-Crime_2022.csv']

In [3]:
# Using read_csv in a list comprehension and combine with concact to load all files
ch_dirt = pd.concat([pd.read_csv(f) for f in glob])

In [4]:
ch_dirt.head()

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude
0,1326041,01/01/2001 01:00:00 AM,BATTERY,SIMPLE,RESIDENCE,False,False,1624,16.0,NaN,41.957850,-87.749185
1,1319931,01/01/2001 01:00:00 PM,BATTERY,SIMPLE,RESIDENCE,False,True,825,8.0,NaN,41.783892,-87.684841
2,1324743,01/01/2001 01:00:00 PM,GAMBLING,ILLEGAL ILL LOTTERY,STREET,True,False,313,3.0,NaN,41.780412,-87.611970
3,1310717,01/01/2001 01:00:00 AM,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,2424,24.0,NaN,42.012391,-87.678032
4,1318099,01/01/2001 01:00:00 AM,BATTERY,SIMPLE,RESIDENCE PORCH/HALLWAY,False,True,214,2.0,NaN,41.819538,-87.620020


In [5]:
ch_dirt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7713109 entries, 0 to 238857
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Date                  object 
 2   Primary Type          object 
 3   Description           object 
 4   Location Description  object 
 5   Arrest                bool   
 6   Domestic              bool   
 7   Beat                  int64  
 8   District              float64
 9   Ward                  float64
 10  Latitude              float64
 11  Longitude             float64
dtypes: bool(2), float64(4), int64(2), object(4)
memory usage: 662.0+ MB


## Performing basic EDA on the dataframe

In [6]:
# inspecting data types for an unexpected data types
ch_dirt.dtypes

ID                        int64
Date                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Latitude                float64
Longitude               float64
dtype: object

All data types make sense for the category

In [7]:
# Checking for any potential dupliacted rows
ch_dirt.duplicated().sum()

0

In [8]:
# Checking for the amount of null values in each collumn
ch_dirt.isna().sum()

ID                           0
Date                         0
Primary Type                 0
Description                  0
Location Description     10928
Arrest                       0
Domestic                     0
Beat                         0
District                    47
Ward                    614846
Latitude                 88685
Longitude                88685
dtype: int64

Many reports are missing location information. Could use district information rather than exact coordinates.

### Inspecting Categorical Columns for any inconsistencies

In [9]:
# Identifying string columns 
string_cols = ch_dirt.select_dtypes('object').columns

In [10]:
# Checking the number of unique values in each categorical column 
ch_dirt[string_cols].nunique()

Date                    3172250
Primary Type                 36
Description                 547
Location Description        215
dtype: int64

In [11]:
# Dropping the Date column as there are too many values
string_cols = string_cols.drop('Date')

In [12]:
# Obtaining value counts for all string columns
for col in string_cols:
    print(f'Value Counts for {col}')
    print(ch_dirt[col].value_counts())
    # Adding extra line for readability
    print('\n')

Value Counts for Primary Type
THEFT                                1626992
BATTERY                              1410889
CRIMINAL DAMAGE                       878914
NARCOTICS                             746155
ASSAULT                               501103
OTHER OFFENSE                         479174
BURGLARY                              422246
MOTOR VEHICLE THEFT                   366586
DECEPTIVE PRACTICE                    340958
ROBBERY                               289677
CRIMINAL TRESPASS                     212869
WEAPONS VIOLATION                     103906
PROSTITUTION                           69750
OFFENSE INVOLVING CHILDREN             55280
PUBLIC PEACE VIOLATION                 52086
SEX OFFENSE                            30442
CRIM SEXUAL ASSAULT                    27584
INTERFERENCE WITH PUBLIC OFFICER       18206
LIQUOR LAW VIOLATION                   14846
GAMBLING                               14616
ARSON                                  13121
HOMICIDE                 

Some things to note. 'Primary Type' has some inconsistencies on how 'non-criminal' is recorded, I will be leaving 'NON-CRIMINAL (SUBJECT SPECIFIED)' as is. Also will replace 'CRIMINAL SEXUAL ASSAULT' with 'CRIM SEXUAL ASSAULT' is the latter is more common. Too many entries for both 'Description' and 'Location Description' to properly look though.

In [13]:
# Setting all 'NON - CRIMINAL' values to ''NON-CRIMINAL'
ch_dirt['Primary Type'] = ch_dirt['Primary Type'].replace({'NON - CRIMINAL': 'NON-CRIMINAL', 'CRIMINAL SEXUAL ASSAULT': 'CRIM SEXUAL ASSAULT'})
# Ensuring the changes went through
ch_dirt['Primary Type'].value_counts()

THEFT                                1626992
BATTERY                              1410889
CRIMINAL DAMAGE                       878914
NARCOTICS                             746155
ASSAULT                               501103
OTHER OFFENSE                         479174
BURGLARY                              422246
MOTOR VEHICLE THEFT                   366586
DECEPTIVE PRACTICE                    340958
ROBBERY                               289677
CRIMINAL TRESPASS                     212869
WEAPONS VIOLATION                     103906
PROSTITUTION                           69750
OFFENSE INVOLVING CHILDREN             55280
PUBLIC PEACE VIOLATION                 52086
CRIM SEXUAL ASSAULT                    33937
SEX OFFENSE                            30442
INTERFERENCE WITH PUBLIC OFFICER       18206
LIQUOR LAW VIOLATION                   14846
GAMBLING                               14616
ARSON                                  13121
HOMICIDE                               12394
KIDNAPPING

In [14]:
# I'll be minimizing bloat by getting rid of each categroy with low crime counts, or redundancies.
ch_dirt['Primary Type'] = ch_dirt['Primary Type'].replace({'DOMESTIC VIOLENCE' : 'OTHER OFFENSE', 
                                                           'NON-CRIMINAL (SUBJECT SPECIFIED)' : 'NON-CRIMINAL',
                                                           'OTHER NARCOTIC VIOLATION' : 'NARCOTICS',
                                                           'RITUALISM' : 'OTHER OFFENSE',
                                                           'PUBLIC INDECENCY' : 'OTHER OFFENSE'})
# Ensuring the changes went through
ch_dirt['Primary Type'].value_counts()

THEFT                                1626992
BATTERY                              1410889
CRIMINAL DAMAGE                       878914
NARCOTICS                             746302
ASSAULT                               501103
OTHER OFFENSE                         479392
BURGLARY                              422246
MOTOR VEHICLE THEFT                   366586
DECEPTIVE PRACTICE                    340958
ROBBERY                               289677
CRIMINAL TRESPASS                     212869
WEAPONS VIOLATION                     103906
PROSTITUTION                           69750
OFFENSE INVOLVING CHILDREN             55280
PUBLIC PEACE VIOLATION                 52086
CRIM SEXUAL ASSAULT                    33937
SEX OFFENSE                            30442
INTERFERENCE WITH PUBLIC OFFICER       18206
LIQUOR LAW VIOLATION                   14846
GAMBLING                               14616
ARSON                                  13121
HOMICIDE                               12394
KIDNAPPING

In [15]:
# Removing the maximum number of displayed rows so I can view all valuecounts
pd.set_option('display.max_rows', None)

In [16]:
# Obtaining value counts for all string columns
for col in string_cols:
    print(f'Value Counts for {col}')
    print(ch_dirt[col].value_counts())
    # Adding extra line for readability
    print('\n')

Value Counts for Primary Type
THEFT                                1626992
BATTERY                              1410889
CRIMINAL DAMAGE                       878914
NARCOTICS                             746302
ASSAULT                               501103
OTHER OFFENSE                         479392
BURGLARY                              422246
MOTOR VEHICLE THEFT                   366586
DECEPTIVE PRACTICE                    340958
ROBBERY                               289677
CRIMINAL TRESPASS                     212869
WEAPONS VIOLATION                     103906
PROSTITUTION                           69750
OFFENSE INVOLVING CHILDREN             55280
PUBLIC PEACE VIOLATION                 52086
CRIM SEXUAL ASSAULT                    33937
SEX OFFENSE                            30442
INTERFERENCE WITH PUBLIC OFFICER       18206
LIQUOR LAW VIOLATION                   14846
GAMBLING                               14616
ARSON                                  13121
HOMICIDE                 

There is far too much variation in the "DESCRIPTION" column for me to ever to make it a usable category while still keeping it accurate to original meaning. I will, However, be trimming 'LOCATION DESCRIPTION' down to a usable size.

In [17]:
# Fixing some inconsistencies in the 'Location Description' column
ch_dirt['Location Description'] = ch_dirt['Location Description'].str.replace('CHA ' , '')
ch_dirt['Location Description'] = ch_dirt['Location Description'].str.replace('CTA ' , '')
ch_dirt['Location Description'] = ch_dirt['Location Description'].str.replace(' / ' , '/')
ch_dirt['Location Description'] = ch_dirt['Location Description'].str.replace(' - ' , '-')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['TAVERN','LIQUOR STORE','BAR', 
                                                                           'BAR OR TAVERN'], 'TAVERN/LIQUOR STORE')
ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['HOTEL', 'MOTEL'], 'HOTEL/MOTEL')
ch_dirt['Location Description'] = ch_dirt['Location Description'].replace({'OTHER (SPECIFY)' : 'OTHER',
                                                                           'TAXICAB' : 'TAXI CAB',
                                                                           'POOLROOM': 'POOL ROOM'})
                                                                          
# Ensuring the changes went through
ch_dirt['Location Description'].value_counts()

STREET                                                 2005166
RESIDENCE                                              1297277
APARTMENT                                               903935
SIDEWALK                                                725870
OTHER                                                   282114
PARKING LOT/GARAGE(NON.RESID.)                          202977
ALLEY                                                   171241
SCHOOL, PUBLIC, BUILDING                                146378
SMALL RETAIL STORE                                      145207
RESIDENCE-GARAGE                                        143239
RESTAURANT                                              125737
RESIDENCE PORCH/HALLWAY                                 124202
VEHICLE NON-COMMERCIAL                                  123084
GROCERY FOOD STORE                                       97969
DEPARTMENT STORE                                         97483
GAS STATION                                            

In [18]:
# Fixing some inconsistencies in the 'Location Description' column
ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['AIRPORT BUILDING NON-TERMINAL-NON-SECURE AREA',
                                                                           'AIRPORT TERMINAL UPPER LEVEL-NON-SECURE AREA',
                                                                           'AIRPORT EXTERIOR-NON-SECURE ARE',
                                                                           'AIRPORT TERMINAL LOWER LEVEL-NON-SECURE AREA',
                                                                           'AIRPORT TERMINAL UPPER LEVEL-SECURE AREA',
                                                                          'AIRPORT TERMINAL LOWER LEVEL-SECURE AREA',
                                                                          'AIRPORT BUILDING NON-TERMINAL-SECURE AREA',
                                                                          'AIRPORT EXTERIOR-SECURE AREA',
                                                                          'AIRPORT TERMINAL MEZZANINE-NON-SECURE AREA',
                                                                           'AIRPORT PARKING LOT',
                                                                           'AIRPORT EXTERIOR-NON-SECURE AREA',
                                                                           'AIRPORT VENDING ESTABLISHMENT ',
                                                                          'AIRCRAFT',
                                                                           'AIRPORT VENDING ESTABLISHMENT',
                                                                           'AIRPORT TRANSPORTATION SYSTEM (ATS)',
                                                                          'AIRPORT'],
                                                                          'AIRPORT/AIRCRAFT')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['SCHOOL-PRIVATE GROUNDS', 
                                                                           'SCHOOL-PRIVATE BUILDING',    
                                                                           'SCHOOL, PUBLIC, BUILDING', 
                                                                           'SCHOOL-PUBLIC BUILDING',
                                                                          'SCHOOL-PUBLIC GROUNDS',
                                                                           'SCHOOL, PUBLIC, GROUNDS',
                                                                           'SCHOOL, PRIVATE, GROUNDS',
                                                                           'SCHOOL, PRIVATE, BUILDING',
                                                                          'SCHOOL YARD',
                                                                          'PUBLIC GRAMMAR SCHOOL',
                                                                           'PUBLIC HIGH SCHOOL',
                                                                           'COLLEGE/UNIVERSITY GROUNDS',
                                                                           'COLLEGE/UNIVERSITY-GROUNDS' ,
                                                                           'COLLEGE/UNIVERSITY-RESIDENCE HALL',
                                                                           'SCHOOL',
                                                                          'COLLEGE/UNIVERSITY RESIDENCE HALL',], 'SCHOOL/COLLEGE/UNIVERSITY')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['STAIRWELL', 
                                                                          'ELEVATOR',
                                                                          'HALLWAY/STAIRWELL/ELEVATOR',
                                                                          ], 'HALLWAY/STAIRWELL/ELEVATOR')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['RESIDENCE-PORCH/HALLWAY', 
                                                                           'RESIDENCE-YARD (FRONT/BACK)',    
                                                                           'RESIDENCE-GARAGE', 
                                                                           'RESIDENTIAL YARD (FRONT/BACK)',
                                                                          'DRIVEWAY-RESIDENTIAL',
                                                                          'HOUSE',
                                                                          'COACH HOUSE',
                                                                           'PARKING LOT/GARAGE (NON RESIDENTIAL)',
                                                                           'RESIDENCE PORCH/HALLWAY',
                                                                           'APARTMENT',
                                                                           'PORCH',
                                                                           'ROOMING HOUSE',
                                                                           'LAUNDRY ROOM',
                                                                           'PROPERTY',
                                                                           'HALLWAY',
                                                                           'BREEZEWAY',
                                                                           'BASEMENT',
                                                                           'TRAILER',
                                                                           'DRIVEWAY'
                                                                           ], 'RESIDENCE')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['CHURCH/SYNAGOGUE/PLACE OF WORSHIP', 
                                                                           'CHURCH PROPERTY',    
                                                                           'CHURCH' 
                                                                           ], 'CHURCH/SYNAGOGUE/PLACE OF WORSHIP')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['VEHICLE-COMMERCIAL', 
                                                                           'VEHICLE NON-COMMERCIAL',    
                                                                           'VEHICLE-OTHER RIDE SHARE SERVICE (LYFT, UBER, ETC.)',
                                                                            'VEHICLE-DELIVERY TRUCK',
                                                                           'TRUCK',
                                                                           'BUS',
                                                                           'DELIVERY TRUCK',
                                                                           'VEHICLE-OTHER RIDE SHARE SERVICE (E.G., UBER, LYFT)',
                                                                           'VEHICLE-OTHER RIDE SERVICE',
                                                                           'VEHICLE-COMMERCIAL-TROLLEY BUS',
                                                                           'VEHICLE-COMMERCIAL: ENTERTAINMENT/PARTY BUS',
                                                                           'VEHICLE-COMMERCIAL-ENTERTAINMENT/PARTY BUS',
                                                                           'OTHER COMMERCIAL TRANSPORTATION',
                                                                           'LIVERY AUTO',
                                                                           'VEHICLE-COMMERCIAL - TROLLEY BUS',
                                                                           'TAXI CAB',
                                                                           'VEHICLE-COMMERCIAL: TROLLEY BUS'
                                                                           ], 'VEHICLE')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['PARKING LOT/GROUNDS', 
                                                                           'PARKING LOT/GARAGE(NON.RESID.)',    
                                                                           'PARKING LOT/GARAGE (NON RESIDENTIAL)',
                                                                            'GARAGE/OTHER PROPERTY',
                                                                           'GARAGE',
                                                                           'PARKING LOT/GARAGE/OTHER PROPERTY'
                                                                           ], 'PARKING LOT')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['OTHER RAILROAD PROP/TRAIN DEPOT', 
                                                                           '"L" TRAIN',    
                                                                           '"L" PLATFORM',
                                                                            'PLATFORM',
                                                                           'TRACKS-RIGHT OF WAY',
                                                                           'STATION',
                                                                           'OTHER RAILROAD PROPERTY/TRAIN DEPOT'
                                                                           ], 'RAILROAD PROPERTY')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['GROCERY FOOD STORE', 
                                                                           'DRUG STORE',    
                                                                           'APPLIANCE STORE',
                                                                            'CLEANING STORE',
                                                                           'SMALL RETAIL STORE',
                                                                           'DEPARTMENT STORE',
                                                                           'CONVENIENCE STORE',
                                                                           'RETAIL STORE',
                                                                           'PAWN SHOP',
                                                                           'AUTO/BOAT/RV DEALERSHIP',
                                                                           'GAS STATION DRIVE/PROP.',
                                                                           'GAS STATION',
                                                                           'CLEANERS/LAUNDROMAT'
                                                                           ], 'COMMERCIAL STORE')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['HOSPITAL BUILDING/GROUNDS', 
                                                                           'MEDICAL/DENTAL OFFICE',    
                                                                           'HOSPITAL',
                                                                           ], 'HOSPITAL/MEDICAL OFFICE')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['HOSPITAL BUILDING/GROUNDS', 
                                                                           'MEDICAL/DENTAL OFFICE',    
                                                                           'HOSPITAL',
                                                                           ], 'HOSPITAL/MEDICAL OFFICE')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['RIVER BANK', 
                                                                           'RIVER',    
                                                                           'LAKE',
                                                                           'LAGOON'
                                                                           ], 'LAKEFRONT/WATERFRONT/RIVERBANK')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['ABANDONED BUILDING', 
                                                                           'VACANT LOT/LAND',    
                                                                           'VACANT LOT',
                                                                           ], 'ABANDONED BUILDING/VACANT LOT')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['CREDIT UNION', 
                                                                           'CURRENCY EXCHANGE',    
                                                                           'BANK',
                                                                           'ATM (AUTOMATIC TELLER MACHINE)',
                                                                           'SAVINGS AND LOAN'
                                                                           ], 'BANK/ATM')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['LIBRARY', 
                                                                           'ATHLETIC CLUB',    
                                                                           'POOL ROOM',
                                                                           'BOWLING ALLEY',
                                                                           'CLUB',
                                                                           'HORSE STABLE',
                                                                           'YMCA',
                                                                           'MOVIE HOUSE/THEATER',
                                                                           'BANQUET HALL'
                                                                           ], 'RECREATION BUILDING')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['YARD', 
                                                                           'GROUNDS',    
                                                                           'VESTIBULE',
                                                                           'CEMETARY',
                                                                           'ALLEY'
                                                                           ], 'ALLEY/GROUNDS')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['COIN OPERATED MACHINE',   
                                                                           'NEWSSTAND',
                                                                           'CEMETARY',
                                                                           'FUNERAL PARLOR',
                                                                           'CAR WASH',
                                                                           'DAY CARE CENTER'
                                                                           ], 'SMALL VENDORS')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace(['SUBWAY STATION', 
                                                                           'TRUCKING TERMINAL',    
                                                                           'LOADING DOCK',
                                                                           'CEMETARY',
                                                                           'JUNK YARD/GARBAGE DUMP',
                                                                           'SEWER',
                                                                           'DUMPSTER',
                                                                           'GANGWAY',
                                                                           'BRIDGE',
                                                                           'FARM','PRAIRIE',   
                                                                           'FOREST PRESERVE',
                                                                           'WOODED AREA'
                                                                           ], 'OTHER')

ch_dirt['Location Description'] = ch_dirt['Location Description'].replace({'VACANT LOT/LAND' : 'VACANT LOT/LAND',
                                                                          'POLICE FACILITY/VEH PARKING LOT' : 'POLICE FACILITY',
                                                                          'POLICE FACILITY/VEHICLE PARKING LOT' : 'POLICE FACILITY',
                                                                           'NURSING HOME' : 'NURSING/RETIREMENT HOME',
                                                                           'GOVERNMENT BUILDING/PROPERTY' : 'GOVERNMENT BUILDING',
                                                                           'BARBERSHOP' : 'BARBER SHOP/BEAUTY SALON',
                                                                           'FEDERAL BUILDING' : 'GOVERNMENT BUILDING',
                                                                           'COMMERCIAL/BUSINESS OFFICE' : 'OFFICE',
                                                                           'LOBBY' : 'OFFICE',
                                                                           'LIVERY STAND OFFICE' : 'OFFICE',
                                                                           'JAIL/LOCK-UP FACILITY' : 'JAIL',
                                                                           'COUNTY JAIL' : 'JAIL',
                                                                           'PLAY LOT' : 'PARK PROPERTY',
                                                                           'KENNEL' : 'KENNEL/ANIMAL HOSPITAL',
                                                                           'ANIMAL HOSPITAL' : 'KENNEL/ANIMAL HOSPITAL',
                                                                           'FACTORY' : 'FACTORY/MANUFACTURING BUILDING',
                                                                           'AUTO' : 'AUTO REPAIR',
                                                                           'GARAGE/AUTO REPAIR': 'AUTO REPAIR',
                                                                           'EXPRESSWAY EMBANKMENT' : 'HIGHWAY/EXPRESSWAY',
                                                                          })
                                                                          
# Ensuring the changes went through
ch_dirt['Location Description'].value_counts()

RESIDENCE                            2606225
STREET                               2005166
SIDEWALK                              725870
COMMERCIAL STORE                      493941
OTHER                                 283108
PARKING LOT                           271379
SCHOOL/COLLEGE/UNIVERSITY             209386
ALLEY/GROUNDS                         171995
VEHICLE                               167546
RESTAURANT                            125737
TAVERN/LIQUOR STORE                    65760
OFFICE                                 59297
PARK PROPERTY                          58407
RAILROAD PROPERTY                      52152
BANK/ATM                               52103
ABANDONED BUILDING/VACANT LOT          38452
HOSPITAL/MEDICAL OFFICE                33944
HOTEL/MOTEL                            33151
AIRPORT/AIRCRAFT                       31854
TRAIN                                  30101
HALLWAY/STAIRWELL/ELEVATOR             25485
POLICE FACILITY                        20966
RECREATION

In [19]:
ch_dirt['Location Description'].nunique()

41

41 unique entries could still very much be considered too large of a cardinality, but I dont believe I can trim it down much more without betraying the data. The rest I can do in Tableau with certain groupings.

In [20]:
# Saving a list of numerical columns
num_cols = ch_dirt.select_dtypes('number').columns
num_cols = num_cols.drop('ID')

In [21]:
# Using .describe to view the statistics of the data
for col in num_cols:# printing a description of each column
    print(f'Value counts for {col}')
    # .apply() being used to suppress scientific notation
    print(ch_dirt[col].describe().apply(lambda x : format(x, 'f')))
    # Print an empty line for readability
    print('\n')
    

Value counts for Beat
count    7713109.000000
mean        1186.293871
std          703.064759
min          111.000000
25%          621.000000
50%         1034.000000
75%         1731.000000
max         2535.000000
Name: Beat, dtype: object


Value counts for District
count    7713062.000000
mean          11.295328
std            6.951157
min            1.000000
25%            6.000000
50%           10.000000
75%           17.000000
max           31.000000
Name: District, dtype: object


Value counts for Ward
count    7098263.000000
mean          22.750796
std           13.847996
min            1.000000
25%           10.000000
50%           23.000000
75%           34.000000
max           50.000000
Name: Ward, dtype: object


Value counts for Latitude
count    7624424.000000
mean          41.842167
std            0.088811
min           36.619446
25%           41.768727
50%           41.855888
75%           41.906766
max           42.022910
Name: Latitude, dtype: object


Value counts for

Everything looks correct, no major outliers in any direction. The beat category is a more specific location so higher variance makes sense. I will not be doing more indepth analysis.

In [22]:
# Setting the maximum number of displayed rows to avoid accidental crashes
pd.set_option('display.max_rows', 30)

## Prepping data for exporting

#### Converting to time series for future splitting

In [51]:
# this cell can take up to 1 min to run
# Initializing the datetime formatting
date_format = "%m/%d/%Y %H:%M:%S %p"
# Implementing the datetime format while also redefining the df as chicago_full
ch_dirt['Date'] = pd.to_datetime(ch_dirt['Date'], format=date_format)
ch_clean = ch_dirt.sort_values('Date')
ch_clean.head()

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude,Datetime
0,1326041,2001-01-01 01:00:00,BATTERY,SIMPLE,RESIDENCE,False,False,1624,16.0,NaN,41.957850,-87.749185,2001-01-01 01:00:00
29,1317937,2001-01-01 01:00:00,BATTERY,SIMPLE,RESIDENCE,False,False,334,3.0,NaN,41.762514,-87.565877,2001-01-01 01:00:00
31,1310824,2001-01-01 01:00:00,THEFT,$500 AND UNDER,TAVERN/LIQUOR STORE,False,False,323,3.0,NaN,41.765484,-87.621836,2001-01-01 01:00:00
32,1313086,2001-01-01 01:00:00,THEFT,OVER $500,OTHER,False,False,1732,17.0,NaN,41.953070,-87.716716,2001-01-01 01:00:00
33,1310858,2001-01-01 01:00:00,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,False,1724,17.0,NaN,41.955619,-87.708919,2001-01-01 01:00:00


In [88]:
# Setting the index to datetime
ch_test = ch_clean.set_index('Datetime')
ch_test

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude
Datetime,,,,,,,,,,,,
2001-01-01 01:00:00,1326041,2001-01-01 01:00:00,BATTERY,SIMPLE,RESIDENCE,False,False,1624,16.0,NaN,41.957850,-87.749185
2001-01-01 01:00:00,1317937,2001-01-01 01:00:00,BATTERY,SIMPLE,RESIDENCE,False,False,334,3.0,NaN,41.762514,-87.565877
2001-01-01 01:00:00,1310824,2001-01-01 01:00:00,THEFT,$500 AND UNDER,TAVERN/LIQUOR STORE,False,False,323,3.0,NaN,41.765484,-87.621836
2001-01-01 01:00:00,1313086,2001-01-01 01:00:00,THEFT,OVER $500,OTHER,False,False,1732,17.0,NaN,41.953070,-87.716716
2001-01-01 01:00:00,1310858,2001-01-01 01:00:00,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,False,1724,17.0,NaN,41.955619,-87.708919
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 12:50:00,12938029,2022-12-31 12:50:00,ASSAULT,SIMPLE,RESIDENCE,False,False,1014,10.0,24.0,41.855911,-87.719966
2022-12-31 12:50:00,12937822,2022-12-31 12:50:00,ASSAULT,AGGRAVATED - HANDGUN,RESIDENCE,False,True,733,7.0,6.0,41.766546,-87.645669
2022-12-31 12:52:00,12937583,2022-12-31 12:52:00,BATTERY,SIMPLE,BARBER SHOP/BEAUTY SALON,False,False,1135,11.0,28.0,41.868829,-87.686098


In [57]:
ch_test['Year'] = ch_test['Date'].dt.year
ch_yearly = ch_test.groupby('Year').apply(lambda x: x.reset_index(drop=True)).reset_index(drop=True)

In [92]:
ch_test['Year'] = ch_test['Date'].dt.year
df_yearly = ch_test.groupby('Year').apply(lambda x: x.reset_index(drop=True)).reset_index(drop=True)
print(df_yearly)

               ID                Date        Primary Type  \
0         1326041 2001-01-01 01:00:00             BATTERY   
1         1317937 2001-01-01 01:00:00             BATTERY   
2         1310824 2001-01-01 01:00:00               THEFT   
3         1313086 2001-01-01 01:00:00               THEFT   
4         1310858 2001-01-01 01:00:00       OTHER OFFENSE   
...           ...                 ...                 ...   
7713104  12938029 2022-12-31 12:50:00             ASSAULT   
7713105  12937822 2022-12-31 12:50:00             ASSAULT   
7713106  12937583 2022-12-31 12:52:00             BATTERY   
7713107  12938420 2022-12-31 12:52:00             ASSAULT   
7713108  12939906 2022-12-31 12:59:00  DECEPTIVE PRACTICE   

                                     Description      Location Description  \
0                                         SIMPLE                 RESIDENCE   
1                                         SIMPLE                 RESIDENCE   
2                                

In [63]:
def split_years(dt):
    dt['year'] = dt['Date'].dt.year
    return [dt[dt['year'] == y] for y in dt['year'].unique()]

In [91]:
year_splt = split_years(ch_yearly)
year_splt

[             ID                Date            Primary Type       Description  \
 0       1326041 2001-01-01 01:00:00                 BATTERY            SIMPLE   
 1       1317937 2001-01-01 01:00:00                 BATTERY            SIMPLE   
 2       1310824 2001-01-01 01:00:00                   THEFT    $500 AND UNDER   
 3       1313086 2001-01-01 01:00:00                   THEFT         OVER $500   
 4       1310858 2001-01-01 01:00:00           OTHER OFFENSE  TELEPHONE THREAT   
 ...         ...                 ...                     ...               ...   
 485881  1922811 2001-12-31 12:50:00                 BATTERY            SIMPLE   
 485882  1916915 2001-12-31 12:51:00           OTHER OFFENSE  TELEPHONE THREAT   
 485883  1916172 2001-12-31 12:51:13           OTHER OFFENSE  TELEPHONE THREAT   
 485884  1921549 2001-12-31 12:55:00                   THEFT         OVER $500   
 485885  1927120 2001-12-31 12:55:00  PUBLIC PEACE VIOLATION       BOMB THREAT   
 
        Locati

In [86]:
year_splt

[             ID                Date            Primary Type       Description  \
 0       1326041 2001-01-01 01:00:00                 BATTERY            SIMPLE   
 1       1317937 2001-01-01 01:00:00                 BATTERY            SIMPLE   
 2       1310824 2001-01-01 01:00:00                   THEFT    $500 AND UNDER   
 3       1313086 2001-01-01 01:00:00                   THEFT         OVER $500   
 4       1310858 2001-01-01 01:00:00           OTHER OFFENSE  TELEPHONE THREAT   
 ...         ...                 ...                     ...               ...   
 485881  1922811 2001-12-31 12:50:00                 BATTERY            SIMPLE   
 485882  1916915 2001-12-31 12:51:00           OTHER OFFENSE  TELEPHONE THREAT   
 485883  1916172 2001-12-31 12:51:13           OTHER OFFENSE  TELEPHONE THREAT   
 485884  1921549 2001-12-31 12:55:00                   THEFT         OVER $500   
 485885  1927120 2001-12-31 12:55:00  PUBLIC PEACE VIOLATION       BOMB THREAT   
 
        Locati

In [99]:
test_splt = pd.DataFrame(year_splt)
test_splt

S:\Users\thoma\anaconda3\envs\dojo-env\lib\site-packages\pandas\core\internals\construction.py:568: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


,0
0,ID Date ...
1,ID Date ...
2,ID Date ...
3,ID Date Pri...
4,ID Date ...
5,ID Date Pri...
6,ID Date Prima...
7,ID Date Pri...
8,ID Date ...
9,ID Date ...


In [94]:
values = np.array([convert(v) for v in values])

NameError: name 'values' is not defined

In [65]:
def df_year(dt):
    return df = pd.DataFrame({'col':L}) for y in dt

,ID,Date,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Latitude,Longitude,Datetime
0,1326041,2001-01-01 01:00:00,BATTERY,SIMPLE,RESIDENCE,False,False,1624,16.0,NaN,41.957850,-87.749185,2001-01-01 01:00:00
29,1317937,2001-01-01 01:00:00,BATTERY,SIMPLE,RESIDENCE,False,False,334,3.0,NaN,41.762514,-87.565877,2001-01-01 01:00:00
31,1310824,2001-01-01 01:00:00,THEFT,$500 AND UNDER,TAVERN/LIQUOR STORE,False,False,323,3.0,NaN,41.765484,-87.621836,2001-01-01 01:00:00
32,1313086,2001-01-01 01:00:00,THEFT,OVER $500,OTHER,False,False,1732,17.0,NaN,41.953070,-87.716716,2001-01-01 01:00:00
33,1310858,2001-01-01 01:00:00,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,False,1724,17.0,NaN,41.955619,-87.708919,2001-01-01 01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238853,12938029,2022-12-31 12:50:00,ASSAULT,SIMPLE,RESIDENCE,False,False,1014,10.0,24.0,41.855911,-87.719966,2022-12-31 12:50:00
238854,12937822,2022-12-31 12:50:00,ASSAULT,AGGRAVATED - HANDGUN,RESIDENCE,False,True,733,7.0,6.0,41.766546,-87.645669,2022-12-31 12:50:00
238855,12937583,2022-12-31 12:52:00,BATTERY,SIMPLE,BARBER SHOP/BEAUTY SALON,False,False,1135,11.0,28.0,41.868829,-87.686098,2022-12-31 12:52:00
238856,12938420,2022-12-31 12:52:00,ASSAULT,SIMPLE,OFFICE,False,False,1432,14.0,32.0,41.930693,-87.685657,2022-12-31 12:52:00


### Data for pandas analysis

In [ ]:
# Defining a new df for use in the other notebook
ch_clean = ch_dirt

In [ ]:
# Dropping unneeded columns to reduce file size
drop_cols = ['Latitude', 'Longitude', 'Ward', 'Beat', 'ID']
ch_clean = ch_clean.drop(columns=drop_cols)
ch_clean.head()
# ch_clean will be used in the other notbook for analysis using pandas

In [ ]:
# Exporting the cleaned dataframe to import later
fpath_out = "Data/clean-chicago-data.csv"
ch_clean.to_csv(fpath_out, index=True)

### Data for analysis in Tableau

In [ ]:
# definining a different df for use in Tableau
ch_tab = ch_dirt

In [ ]:
# Dropping unneeded columns to reduce file size
drop_cols = ['Beat', 'ID']
ch_tab = ch_tab.drop(columns=drop_cols)
ch_tab.head()
# No need for the 'ID' column in Tableau. The 'Ward' column is being dropped due to the cardinality and overall difficulty to understand.

In [ ]:
# Exporting for use in Tableau
fpath_out = "Data/chicago-data-for-tableau.csv"
ch_tab.to_csv(fpath_out, index=True)